In [45]:

import pandas as pd
import json

In [ ]:
folder_path = '/mnt/d/CollegeProject/UPNepse/data_preparation'
comp_name = "NICA"

In [242]:
def get_cleaned_f_data():
    with open('/mnt/d/CollegeProject/UPNepse/data_preparation/financial_data_transform/data/NICA/financials.json','r') as fj:

        fin_json = json.load(fj)
    df = pd.DataFrame(fin_json)
    df['start_date'] = pd.to_datetime(df['start_date'],unit='ms')
    df['end_date'] = pd.to_datetime(df['end_date'],unit='ms')
    df.drop('fiscal_year',axis=1,inplace=True)
    for col in df.columns:
            if  col not in ['symbol','start_date','end_date']:
                    df[col] = df[col].astype(float)

    zero_columns = df.columns[df.eq(0).all()]
    df.drop(zero_columns,axis=1,inplace=True)
    zero_counts = (df == 0).sum()
    columns_to_drop = zero_counts[zero_counts >= 3].index
    df.drop(columns_to_drop,axis=1,inplace=True)
    df.sort_values(by='start_date',inplace=True)
    df.reset_index(drop=True,inplace=True)


    new_rows = []
    for idx, row in df.iterrows():
        dates = pd.date_range(start=row['start_date'], end=row['end_date'])
        column_to_remove = ['end_date']

        values = [row[col] for col in df.columns if col not in ['start_date', 'end_date']]
        new_columns = [col for col in df.columns if col not in column_to_remove]
        for date in dates:
                new_row = {col: value for col, value in zip(new_columns, [date] + values)}

                new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows)
    return new_df

In [237]:
print(get_cleaned_f_data())

     start_date  CashAndCashEquivalent  DueFromNepalRastraBank  \
0    2016-07-16           3.549208e+09            4.396160e+09   
1    2016-07-17           3.549208e+09            4.396160e+09   
2    2016-07-18           3.549208e+09            4.396160e+09   
3    2016-07-19           3.549208e+09            4.396160e+09   
4    2016-07-20           3.549208e+09            4.396160e+09   
...         ...                    ...                     ...   
2550 2023-07-12           2.781332e+10            1.265449e+10   
2551 2023-07-13           2.781332e+10            1.265449e+10   
2552 2023-07-14           2.781332e+10            1.265449e+10   
2553 2023-07-15           2.781332e+10            1.265449e+10   
2554 2023-07-16           2.781332e+10            1.265449e+10   

      DerivativeFinancialInstruments  OtherTradingAssets  \
0                       2.266526e+09                 0.0   
1                       2.266526e+09                 0.0   
2                       2.2

In [243]:
def get_cleaned_d_data():
    with open('/mnt/d/CollegeProject/UPNepse/data_preparation/dividend_data_transform/data/NICA/dividend.json','r') as fj:
        data_json = json.load(fj)
    
    dividend_df = pd.DataFrame(data_json)
    dividend_df['book_closure_date']
    
    change_col = ['bonus_dividend','cash_dividend']
    dividend_df[change_col] = dividend_df[change_col].astype(float)
    dividend_df['start_date'] = pd.to_datetime(dividend_df['start_date'],unit='ms')
    dividend_df['end_date'] = pd.to_datetime(dividend_df['end_date'],unit='ms')
    dividend_df['book_closure_date'] = pd.to_datetime(dividend_df['book_closure_date'])
    dividend_df.drop('fiscal_year',axis=1,inplace=True)
    dividend_df.drop('dividend_growth_rate',axis=1,inplace=True)
    dividend_df.sort_values(by='start_date',inplace=True)
    dividend_df.reset_index(drop=True,inplace=True)
    new_dividend_rows = []
    for idx, row in dividend_df.iterrows():
        dates = pd.date_range(start=row['start_date'], end=row['end_date'])
        column_to_remove = ['end_date']
    
        values = [row[col] for col in dividend_df.columns if col not in ['start_date', 'end_date']]
        new_columns = [col for col in dividend_df.columns if col not in column_to_remove]
        for date in dates:
                new_row = {col: value for col, value in zip(new_columns, [date] + values)}
    
                new_dividend_rows.append(new_row)
        
    new_dividend_df = pd.DataFrame(new_dividend_rows)
    new_dividend_df['book_closure_flag'] = 0
    new_dividend_df.loc[new_dividend_df['start_date'] == new_dividend_df['book_closure_date'], 'book_closure_flag'] = 1
    new_dividend_df.drop(['book_closure_date','symbol'],axis=1,inplace=True)

    return new_dividend_df

In [239]:
print(get_cleaned_d_data())

     start_date  bonus_dividend  cash_dividend  book_closure_flag
0    2012-07-16             0.0           25.0                  0
1    2012-07-17             0.0           25.0                  0
2    2012-07-18             0.0           25.0                  0
3    2012-07-19             0.0           25.0                  0
4    2012-07-20             0.0           25.0                  0
...         ...             ...            ...                ...
3645 2024-07-11            29.0            1.5                  0
3646 2024-07-12            29.0            1.5                  0
3647 2024-07-13            29.0            1.5                  0
3648 2024-07-14            29.0            1.5                  0
3649 2024-07-15            29.0            1.5                  0

[3650 rows x 4 columns]


In [244]:
def get_cleaned_stock_data():

    stock_df = pd.read_parquet('/mnt/d/CollegeProject/UPNepse/data_preparation/stock_data_preparation/data/NICA.parquet')
    stock_df.fillna(method='bfill', inplace=True)
    stock_df.fillna(method='ffill', inplace=True)
    return stock_df
    

In [241]:
print(get_cleaned_stock_data())

            Date   Close    Open    High     Low   Volume    SMA_20   SMA_100  \
0     2013-07-14   90.14  128.26   90.14   85.96     40.0  118.2035  114.3163   
1     2013-07-15   97.89   90.14   97.89   94.02     40.0  118.2035  114.3163   
2     2013-07-17  106.41   97.89  106.41   99.59    150.0  118.2035  114.3163   
3     2013-07-18  115.71  106.41  115.71  108.27   1706.0  118.2035  114.3163   
4     2013-07-21  125.94  115.71  125.94  117.72  12269.0  118.2035  114.3163   
...          ...     ...     ...     ...     ...      ...       ...       ...   
2431  2024-03-12  433.50  428.40  434.00  422.00  85533.0  427.3850  494.4043   
2432  2024-03-13  432.00  442.10  442.10  428.00  84991.0  426.4800  492.7941   
2433  2024-03-14  425.10  432.00  432.90  425.10  67871.0  425.6400  491.1420   
2434  2024-03-17  418.00  424.20  424.20  417.00  58774.0  424.3900  489.5920   
2435  2024-03-18  418.20  416.00  421.50  416.00  45079.0  423.6500  488.0940   

          EMA_20     EMA_10

In [282]:
def merge_all_data():
    fdf = get_cleaned_f_data()
    fdf.rename(columns={'start_date':'date'},inplace=True)
    fdf['date'] = pd.to_datetime(fdf['date'])
    ddf = get_cleaned_d_data()
    ddf.rename(columns={'start_date':'date'},inplace=True)
    ddf['date'] = pd.to_datetime(ddf['date'])
    sdf = get_cleaned_stock_data()
    sdf.rename(columns={'Date':'date'},inplace=True)
    sdf['date'] = pd.to_datetime(sdf['date'])
    with open('/mnt/d/CollegeProject/UPNepse/data_preparation/news_data_transform/data/news.json','r') as nf:
        news_json = json.load(nf)
        for news in news_json:
            if 'NICA' in news:
                news_df = pd.DataFrame({'date':news['NICA']})
                news_df['date'] = pd.to_datetime(news_df['date'])


    
    merged_df = pd.merge(sdf, fdf, on='date', how='left') 
    merged_df = pd.merge(merged_df, ddf, on='date', how='left') 
    merged_df['news_flag'] = merged_df['date'].isin(news_df['date']).astype(int)

    merged_df = merged_df[merged_df['date'] >= pd.to_datetime('2016-07-14')]
    merged_df['CashAndCashEquivalent']  = merged_df['CashAndCashEquivalent'].astype(float)
    merged_df['bonus_dividend']  = merged_df['bonus_dividend'].astype(float)
    merged_df['symbol'] = merged_df['symbol'].astype(str)
    merged_df['symbol'].fillna(method='bfill',inplace=True)
    columns_with_missing_values = merged_df.columns[merged_df.isnull().any()].tolist()
    for col in columns_with_missing_values:
        merged_df[col] = merged_df[col].interpolate()


    merged_df.dropna(inplace=True)
    print(merged_df)
merge_all_data()
    

           date   Close    Open    High     Low   Volume    SMA_20   SMA_100  \
677  2016-07-17  260.98  252.93  261.29  252.00  32592.0  248.8720  225.4421   
678  2016-07-18  278.32  260.98  286.37  265.93  56056.0  251.1015  226.3140   
679  2016-07-19  283.89  283.58  287.61  273.05  80145.0  253.5630  227.2168   
680  2016-07-20  285.75  287.61  294.73  277.70  84210.0  256.0245  228.0986   
681  2016-07-21  281.41  285.75  290.39  278.32  43448.0  258.0525  228.9222   
...         ...     ...     ...     ...     ...      ...       ...       ...   
2431 2024-03-12  433.50  428.40  434.00  422.00  85533.0  427.3850  494.4043   
2432 2024-03-13  432.00  442.10  442.10  428.00  84991.0  426.4800  492.7941   
2433 2024-03-14  425.10  432.00  432.90  425.10  67871.0  425.6400  491.1420   
2434 2024-03-17  418.00  424.20  424.20  417.00  58774.0  424.3900  489.5920   
2435 2024-03-18  418.20  416.00  421.50  416.00  45079.0  423.6500  488.0940   

          EMA_20     EMA_100  ...  C.To

In [281]:
merge_all_data()

date                 False
Close                False
Open                 False
High                 False
Low                  False
                     ...  
No.OfAtms            False
bonus_dividend       False
cash_dividend        False
book_closure_flag    False
news_flag            False
Length: 79, dtype: bool


In [291]:

df = pd.read_parquet('data/final_dataset.parquet')
print(df)

             date    Close     Open     High      Low  Volume   SMA_20  \
0      2020-07-29   373.55   341.66   373.55   341.66    80.0  519.024   
1      2020-07-30   410.76   380.39   410.76   380.39    90.0  519.024   
2      2020-08-02   451.76   418.35   451.76   418.35   230.0  519.024   
3      2020-08-03   496.55   460.11   496.55   460.11   776.0  519.024   
4      2020-08-05   545.90   506.42   545.90   506.42  4464.0  519.024   
...           ...      ...      ...      ...      ...     ...      ...   
168298 2024-03-12   955.90   886.00   955.90   886.00  4317.0  822.850   
168299 2024-03-13  1051.40   975.00  1051.40   975.00  2925.0  832.370   
168300 2024-03-14  1156.50  1072.40  1156.50  1072.40  3747.0  846.645   
168301 2024-03-17  1066.00  1179.60  1248.00  1060.00  8314.0  857.445   
168302 2024-03-18  1064.80  1087.30  1107.00   984.00  2234.0  868.530   

         SMA_100      EMA_20     EMA_100  ...  \
0       913.3302  519.024000  913.330200  ...   
1       913.3